[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fuenfgeld/DMA2022DataProjectB/blob/main/BreastCancer.ipynb)

# Laden die Brustkrebs in die sqlite-Databenk

In [1]:
#Laden Libraries
import sqlite3
import pandas as pd

# Laden GitHub repo
#%cd /content/
%rm -r ./DMA2022DataProjectB
!git clone https://github.com/Fuenfgeld/DMA2022DataProjectB.git
    
%cd ./DMA2022DataProjectB/

rm: ./DMA2022DataProjectB: No such file or directory
Cloning into 'DMA2022DataProjectB'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 75 (delta 8), reused 9 (delta 3), pack-reused 55
Receiving objects: 100% (75/75), 2.07 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (30/30), done.
/Volumes/Macintosh HD - Data/Users/mariiarobota/My documents/Studium/Master/DAF/DMA2022DataProjectB/DMA2022DataProjectB


In [2]:
# Verbindung zur Datenbank
conn = sqlite3.connect('breast_cancer.db')
cursor = conn.cursor()
print("Opened database successfully");

Opened database successfully


In [3]:
# Löschen Tabellen wenn sie schon exestieren
cursor.execute("DROP TABLE IF EXISTS patient")
cursor.execute("DROP TABLE IF EXISTS condition")
cursor.execute("DROP TABLE IF EXISTS medication")
cursor.execute("DROP TABLE IF EXISTS observation")
cursor.execute("DROP TABLE IF EXISTS procedures")

In [4]:
# Laden Daten
patient = pd.read_csv('BreastCancerData/patients.csv', sep=",")
condition = pd.read_csv('BreastCancerData/conditions.csv', sep=",")
medication = pd.read_csv('BreastCancerData/medications.csv', sep=",")
observation = pd.read_csv('BreastCancerData/observations.csv', sep=",")
procedure = pd.read_csv('BreastCancerData/procedures.csv', sep=",")

In [5]:
# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient (
			Id nvarchar(36) primary key,
            BIRTHDATE Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            DRIVERS nvarchar(50),
            PASSPORT nvarchar(10),
            PREFIX nvarchar(3),
            FIRST nvarchar(50),
            LAST nvarchar(50),
            SUFFIX nvarchar(50),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(20),
            GENDER nvarchar(2),
            BIRTHPLACE nvarchar(50),
            ADDRESS nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
            
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS condition (
            START Date,
            STOP Date,
            PATIENT int,
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(50),
            CODE int,
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medication (
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT int,
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observation (
            DATE nvarchar(20),
            PATIENT int,
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            VALUE float,
            UNITS nvarchar(10),
            TYPE nvarchar(10),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedure (
            DATE nvarchar(20),
            PATIENT int,
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patient(Id) 
			)
               ''')


In [6]:
#insert data into table
patient.to_sql('patient', conn, if_exists='append', index=False)
condition.to_sql('condition', conn, if_exists='append', index=False)
medication.to_sql('medication', conn, if_exists='append', index=False)
observation.to_sql('observation', conn, if_exists='append', index=False)
procedure.to_sql('procedure', conn, if_exists='append', index=False)

In [7]:
procedure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   DATE               746 non-null    object 
 1   PATIENT            746 non-null    object 
 2   ENCOUNTER          746 non-null    object 
 3   CODE               746 non-null    int64  
 4   DESCRIPTION        746 non-null    object 
 5   BASE_COST          746 non-null    float64
 6   REASONCODE         710 non-null    float64
 7   REASONDESCRIPTION  710 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 46.8+ KB


In [8]:
conn.close()